# 2.2 Inter Process Communication (IPC) using Message Passing:

- In message passing, processes communicate by sending and receiving messages, typically through queues, pipes, or sockets. This method does not require shared memory, and each process has its own memory space.

### Why we use Message Passing for IPC?
- Decoupled Communication: Each process operates independently, without sharing memory, reducing potential synchronization issues (process synchronization).
- Cross-Machine Communication: Enables communication between processes running on different machines or systems, unlike shared memory, which is confined to a single machine.
- Scalability: Suitable for large, distributed systems as it supports communication between processes on different nodes.
- Security and Isolation: As processes do not share memory, message passing can provide better isolation, reducing risks of data corruption or unintended access.

### a. Process Communication Using Connection Object
- The multiprocessing.Connection object is a part of the multiprocessing module in Python, used for inter-process communication (IPC). It provides a simple, low-level API for sending and receiving data between processes, enabling seamless data exchange.

### What is a Connection Object?
- A Connection object represents one end of a Pipe created using the multiprocessing.Pipe() function.
- It offers methods to send and receive data, check for incoming messages, and close the connection.
- Connection objects are integral to process communication because they handle pickling and unpickling of data, making it easy to share complex Python objects.

### Methods of Connection Object:
#### i. conn.send(obj):
- Sends a picklable Python object through the connection.
- Blocks the sending process until the receiving process reads the data if the pipe is full.
#### ii. conn.recv():
- Receives an object sent by the other end of the connection.
- Blocks until data is available to read.
- Raises an EOFError if the other end of the connection is closed and no more data is available.
#### iii. conn.close():
- Closes the connection end, freeing up system resources.
- Once closed, the connection cannot be reopened.
#### iv. conn.poll(timeout=None):
- Non-blocking method to check if there is data available to be read.
- Returns True if data is ready to be received, otherwise False.
- Accepts an optional timeout parameter in seconds.
#### v. conn.fileno():
- Returns the file descriptor or handle associated with the connection.
- Useful for advanced use cases like integrating with select for event-driven communication.
#### iv. conn.is_closed ():
- Checks if the connection is closed.
- Returns True if the connection is closed, otherwise False.

## Message Passing Data Structures:

### i. IPC Using Pipe:
- A Pipe in Python’s multiprocessing module is a lightweight mechanism for inter-process communication (IPC). It allows two processes to exchange data directly via a pair of connection objects, representing the two ends of the pipe. Pipes are especially useful for simple, point-to-point communication between a parent and child process or two related processes.

### Pipe Types
#### i. Full Duplex Pipe (default):
- Both ends (conn1 and conn2) can send and receive data.
- Example: conn1.send(data) and conn2.recv().
#### ii. Half-Duplex Pipe:
- One end only sends, and the other only receives.
- Created by setting duplex=False during pipe creation.

### Workflow of Pipe:
- i. Create Pipe: Use Pipe() to create two connections: conn1 (receiver) and conn2 (sender).
- ii. Sender: Uses conn2.send(data) to send data, then closes conn2 after sending.
- iii. Receiver: Uses conn1.recv() to receive data, then closes conn1 after processing.
- iv. Process Execution: Start both sender and receiver processes, and ensure they complete using join().
- v. Communication: Data flows from conn2 (sender) to conn1 (receiver).

This is the case fo half-duplex and in case of full-duplex you can use same conncetion object to send and receive for a process.

### i. Full Duplex Pipe:

In [1]:
import multiprocessing
import time

def child_process(conn):
    print("[Child] Waiting for a message from parent...")
    message = conn.recv()  # Receive message from parent
    print(f"[Child] Received: {message}")

    response = "Hello from Child!"
    time.sleep(2)  # Simulate some work
    conn.send(response)  # Send response back to parent
    print("[Child] Response sent!")

if __name__ == "__main__":
    parent_conn, child_conn = multiprocessing.Pipe()  # Create a duplex pipe

    process = multiprocessing.Process(target=child_process, args=(child_conn,))
    process.start()

    # Parent process sends a message
    print("[Parent] Sending a message to child...")
    parent_conn.send("Hello from Parent!")

    # Wait for a response
    response = parent_conn.recv()
    print(f"[Parent] Received: {response}")

    process.join()  # Wait for the child process to complete
    print("Communication complete!")


[Child] Waiting for a message from parent...
[Child] Received: Hello from Parent!
[Parent] Sending a message to child...
[Child] Response sent!
[Parent] Received: Hello from Child!
Communication complete!


### ii. Half-Duplex Pipe:

In [ ]:
from multiprocessing import Process, Pipe

def sender(conn):
    conn.send("Hello from sender!")  # Send a message
    conn.close()  # Close the write-only connection

def receiver(conn):
    message = conn.recv()  # Receive a message
    print(f"Received: {message}")
    conn.close()  # Close the read-only connection

if __name__ == "__main__":
    recv_conn, sender_conn = Pipe(duplex=False)  # Create a half-duplex pipe

    p1 = Process(target=sender, args=(sender_conn,))
    p2 = Process(target=receiver, args=(recv_conn,))

    p1.start()
    p2.start()

    p1.join()
    p2.join()


Received: Hello from sender!


### Advantages of Pipe Over Queue:
- Lightweight Communication: Pipes are simpler and have less overhead compared to queues, making them faster for small-scale communication.
- Low Latency: Pipes provide faster message transfer due to their direct communication mechanism.
- Direct Process Connection: Ideal for point-to-point communication between two processes without additional management layers.

### Limitations of Pipe Over Queue:
- Limited to Two Processes: Pipes are designed for one-to-one communication and are not well-suited for multiple producers or consumers.
- Lack of Data Buffering: Unlike queues, pipes do not provide built-in buffering or task queuing mechanisms.
- No Thread Safety: Pipes are not inherently thread-safe, whereas queues provide thread-safe communication.

### Use Cases for Pipe:
- Simple Process-to-Process Communication: Sharing small data or messages between two processes.
- Real-time Data Exchange: Scenarios requiring minimal delay, such as sensor data processing.
- Lightweight Applications: Programs with limited inter-process communication needs.

### ii. IPC Using Queue:
- A Queue is a FIFO (First-In-First-Out) data structure used for inter-process communication (IPC) in Python. It facilitates safe data sharing between multiple producer and consumer processes.


### Key Methods of Queue:
#### i. put(item):
- Adds an item to the queue.
- Blocks if the queue is full unless a timeout is specified.
#### ii. get():
- Retrieves and removes the next item from the queue.
- Blocks if the queue is empty unless a timeout is specified.
#### iv. qsize():
- Returns the number of items in the queue.
- Note: This value may not be accurate in multithreaded environments.
#### v. empty():
- Returns True if the queue is empty; otherwise, False.
#### v. full():
- Returns True if the queue is full; otherwise, False.
#### vi. close():

- Closes the queue and releases resources.


### Workflow of Queue:
#### i. Create Queue:
- Use multiprocessing.Queue(maxsize=0) to create a queue object.
- maxsize determines the maximum number of items allowed in the queue (0 means unlimited).
#### ii. Producer Process:
- Uses queue.put(item) to add items to the queue.
#### iii. Consumer Process:
- Uses queue.get() to retrieve and process items from the queue.
#### v. Communication:
- Multiple producer and consumer processes can share the same queue.

#### Synchronization:
- The queue internally uses locks to ensure thread-safe operations.

In [ ]:
from multiprocessing import Process, Queue

def producer(queue):
    for i in range(5):
        print(f"Producer: Adding {i} to queue")
        queue.put(i)

def consumer(queue):
    while not queue.empty():
        item = queue.get()
        print(f"Consumer: Retrieved {item} from queue")

if __name__ == "__main__":
    queue = Queue()

    producer_process = Process(target=producer, args=(queue,))
    consumer_process = Process(target=consumer, args=(queue,))

    producer_process.start()
    producer_process.join()

    consumer_process.start()
    consumer_process.join()


### Advantages of Queue over Pipe:
- Multiple Producers/Consumers: Queue supports multiple processes sending and receiving data.
- Thread-Safe: Built-in synchronization for safe concurrent access.
- Buffering: Stores data temporarily, preventing data loss.

### Limitations of Queue over Pipe:
- Overhead: Slightly higher overhead due to synchronization and data management.
- Memory Consumption: Requires more memory for internal buffering.
- Latency: May introduce higher latency compared to Pipe for simple communication

### Use Cases of Queue:
- Task Distribution: Distributing jobs to worker processes in parallel systems.
- Data Pipelines: Passing data between processing stages in workflows.
- Batch Processing: Managing work in batches for analysis or computation.
- Parallel Algorithms: Coordinating tasks in multi-process environments.

### Limitations of Message Passing in IPC:
- Overhead: Serialization/deserialization adds extra overhead.
- Latency: Higher latency, especially for network communication.
- Complexity: Managing message queues in large systems can be complex.
- Limited Bandwidth: Data transfer size can be limited.